## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Patricio,US,28.0170,-97.5169,80.85,90,56,13.65,broken clouds
1,1,Broome,US,42.2506,-75.8330,51.17,82,94,1.01,overcast clouds
2,2,Hella,NO,67.2000,14.4333,43.88,81,40,12.66,scattered clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,65.03,77,20,3.20,few clouds
4,4,Talnakh,RU,69.4865,88.3972,27.86,71,100,1.95,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Patricio,US,28.0170,-97.5169,80.85,90,56,13.65,broken clouds
5,5,Hithadhoo,MV,-0.6000,73.0833,81.84,79,100,18.99,overcast clouds
7,7,Buloh Kasap,MY,2.5536,102.7640,84.11,68,72,2.89,broken clouds
8,8,Hirara,JP,24.8000,125.2833,88.07,79,75,23.02,broken clouds
9,9,Bubaque,GW,11.2833,-15.8333,77.56,79,14,8.57,few clouds
11,11,Butaritari,KI,3.0707,172.7902,83.66,78,65,15.23,light rain
13,13,Kapaa,US,22.0752,-159.3190,80.11,69,49,3.00,scattered clouds
19,19,Baculin,PH,7.4525,126.5842,85.48,73,27,3.85,scattered clouds
28,28,Isangel,VU,-19.5500,169.2667,75.81,51,100,3.06,overcast clouds
29,29,Vaini,TO,-21.2000,-175.2000,77.16,73,20,8.34,few clouds


In [5]:
# 4. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Country                213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Patricio,US,80.85,broken clouds,28.0170,-97.5169,
5,Hithadhoo,MV,81.84,overcast clouds,-0.6000,73.0833,
7,Buloh Kasap,MY,84.11,broken clouds,2.5536,102.7640,
8,Hirara,JP,88.07,broken clouds,24.8000,125.2833,
9,Bubaque,GW,77.56,few clouds,11.2833,-15.8333,
11,Butaritari,KI,83.66,light rain,3.0707,172.7902,
13,Kapaa,US,80.11,scattered clouds,22.0752,-159.3190,
19,Baculin,PH,85.48,scattered clouds,7.4525,126.5842,
28,Isangel,VU,75.81,overcast clouds,-19.5500,169.2667,
29,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ''].index, inplace = False)
clean_hotel_df.shape

(204, 7)

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))